<a href="https://colab.research.google.com/github/No-Qubit-Left-Behind/Control-Engineering-in-TF/blob/master/TF_Propagator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Propagator

In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import time
print(tf.__version__)

2.1.0


In [7]:
class Propagator:
    def __init__(self, no_of_steps, dim):
        self.dim = dim
        
        # generate two random Hermiatian matrices x & y
        x = tf.complex(
            tf.random.uniform([self.dim, self.dim], -1, 1, dtype=tf.float64),
            tf.random.uniform([self.dim, self.dim], -1, 1, dtype=tf.float64)
        )
        y = tf.complex(
            tf.random.uniform([self.dim, self.dim], -1, 1, dtype=tf.float64),
            tf.random.uniform([self.dim, self.dim], -1, 1, dtype=tf.float64)
        )
        x = 1. / 2. * (x - tf.linalg.adjoint(x))
        y = 1. / 2. * (y - tf.linalg.adjoint(y))

        self.generators =  tf.stack([x, y])
        self.ctrl_amplitudes = tf.Variable(
            tf.zeros([no_of_steps, 2], dtype=tf.complex128),
            dtype=tf.complex128
        )

        """
            self.contraction_array determines the neccessity for the extra
            matrix multiplication step in the recursive method self.propagate()
            when the intermediate computation array has length not divisible
            by 2
        """
        self.contraction_array = []
        contraction_array_length = int(np.floor(np.log2(no_of_steps)))
        temp_no_of_steps = no_of_steps
        for i in range(contraction_array_length):
            self.contraction_array.append(bool(np.mod(temp_no_of_steps, 2)))
            temp_no_of_steps = np.floor(temp_no_of_steps / 2)
  
    """
        exponentials() computes a vector matrix exponential after multiplying
        each self.ctrl_amplitudes row with a the vector of matrices in
        self.generators
    """
    def exponentials(self):
        exponents = tf.linalg.tensordot(
            self.ctrl_amplitudes, self.generators, 1
        )
        return tf.linalg.expm(exponents)

     
    """
        propagate  computes the final propagator by recursively multiplying
        each odd element in the list of matrices with each even element --
        if the length of the array is not divisible by 2 an extra computation
        step is added
    """
    def propagate(self):
        step_exps = self.exponentials()
        for is_odd in self.contraction_array:
            if is_odd:
                odd_exp = step_exps[-1, :, :]
                step_exps = tf.linalg.matmul(
                    step_exps[1::2, :, :], step_exps[0:-1:2, :, :]
                )
                step_exps = tf.concat([
                    step_exps[0:-1, :, :],
                    [tf.linalg.matmul(odd_exp, step_exps[-1, :, :])]
                ], 0)
            else:
                step_exps = tf.linalg.matmul(
                    step_exps[1::2, :, :], step_exps[0::2, :, :]
                )
        return tf.squeeze(step_exps)

    """
        __call__ computes the final propagator fidelity squared with the
        identity operator
    """
    @tf.function
    def __call__(self):
        tr = tf.linalg.trace(self.propagate())
        return 1 - tf.math.real(tr * tf.math.conj(tr)) / (self.dim ** 2)

propagator = Propagator(10000, 20)

start = time.time()
fidelity = propagator()
with tf.GradientTape() as tape:
    fidelity = propagator()
gradients = tape.gradient(fidelity, [propagator.ctrl_amplitudes])

end = time.time()
print(end-start)

6.882051467895508
